# Code execution and the Files API

The Anthropic API offers two powerful features that work exceptionally well together: the Files API and Code Execution. While they might seem separate at first, combining them opens up some really interesting possibilities for delegating complex tasks to Claude.

In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic
from pathlib import Path

load_dotenv()

client = Anthropic(
    default_headers={
        "anthropic-beta": "code-execution-2025-08-25, files-api-2025-04-14"
    }
)
model = "claude-sonnet-4-5-20250929"

In [2]:
# Helper functions
from anthropic.types import Message


def add_messages(messages, message, role="user"):
    message = {"role": role, "content": message.content if isinstance(message, Message) else message}
    messages.append(message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=2000,
):
    params = {
        "model": model,
        "max_tokens": 10000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])


def upload(file_path):
    path = Path(file_path)
    extension = path.suffix.lower()

    mime_type_map = {
        ".pdf": "application/pdf",
        ".txt": "text/plain",
        ".md": "text/plain",
        ".py": "text/plain",
        ".js": "text/plain",
        ".html": "text/plain",
        ".css": "text/plain",
        ".csv": "text/csv",
        ".json": "application/json",
        ".xml": "application/xml",
        ".xlsx": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        ".xls": "application/vnd.ms-excel",
        ".jpeg": "image/jpeg",
        ".jpg": "image/jpeg",
        ".png": "image/png",
        ".gif": "image/gif",
        ".webp": "image/webp",
    }

    mime_type = mime_type_map.get(extension)

    if not mime_type:
        raise ValueError(f"Unknown mimetype for extension: {extension}")
    filename = path.name

    with open(file_path, "rb") as file:
        return client.beta.files.upload(file=(filename, file, mime_type))


def list_files():
    return client.beta.files.list()


def delete_file(id):
    return client.beta.files.delete(id)


def download_file(id, filename=None):
    file_content = client.beta.files.download(id)

    if not filename:
        file_metadata = get_metadata(id)
        file_content.write_to_file(file_metadata.filename)
    else:
        file_content.write_to_file(filename)


def get_metadata(id):
    return client.beta.files.retrieve_metadata(id)

In [3]:
file_metadata = upload("streaming.csv")
file_metadata

FileMetadata(id='file_011CYD255YT9oK9WvruntC5B', created_at=datetime.datetime(2026, 2, 17, 4, 35, 21, 844000, tzinfo=datetime.timezone.utc), filename='streaming.csv', mime_type='text/csv', size_bytes=25733, type='file', downloadable=False)

In [4]:
messages = []

add_messages(
    messages,
    [
        {
            "type": "text",
            "text": """
Run a detailed analysis to determine major drivers of churn.
Your final output should include at least one detailed plot summarizing your findings.

Critical note: Every time you execute code, you're starting with a completely clean slate. 
No variables or library imports from previous executions exist. You need to redeclare/reimport all variables/libraries.
            """,
        },
        {"type": "container_upload", "file_id": file_metadata.id},
    ],
    role="user",
)

chat(messages, tools=[{"type": "code_execution_20250825", "name": "code_execution"}])

Message(id='msg_01VFNKRvJPZwvfLxwvXkjQgd', content=[TextBlock(citations=None, text="I'll help you analyze the drivers of churn in the streaming dataset. Let me start by exploring the data and then perform a comprehensive analysis.", type='text'), ServerToolUseBlock(id='srvtoolu_01V4QqXpWpbzPsahYdwuJx1f', input={'command': 'cd $INPUT_DIR && ls -lh'}, name='bash_code_execution', type='server_tool_use', caller={'type': 'direct'}), TextBlock(citations=None, text=None, type='bash_code_execution_tool_result', tool_use_id='srvtoolu_01V4QqXpWpbzPsahYdwuJx1f', content={'type': 'bash_code_execution_result', 'stdout': 'total 26K\n-rw-r--r-- 1 root root 26K Feb 17 04:41 streaming.csv\n', 'stderr': '', 'return_code': 0, 'content': []}), ServerToolUseBlock(id='srvtoolu_01AdtKRz3i5AyqXiixKbqMoj', input={'command': 'view', 'path': '$INPUT_DIR/streaming.csv', 'view_range': [1, 50]}, name='text_editor_code_execution', type='server_tool_use', caller={'type': 'direct'}), TextBlock(citations=None, text=Non

In [5]:
download_file("file_011CYD2d8kpkJkLXxdKRUXWL")